Tutorial based on this [page](https://realpython.com/factory-method-python/)

# Goal 

let's work with an application that needs to convert a **Song** object into its string representation (using a specified format). 

__serialization__: when converting an object to a different representation



# Traditional way to do this


In [2]:
import json
import xml.etree.ElementTree as et

In [7]:
class Song:
    def __init__(self, song_id, title, artist):
        self.song_id = song_id
        self.title = title
        self.artist = artist

class SongSerializer:
    def serialize(self, song, format):
        if format == 'JSON':
            song_info = {
                'id': song.song_id,
                'title': song.title,
                'artist': song.artist
            }
            return json.dumps(song_info)
        elif format == 'XML':
            song_info = et.Element('song', attrib={'id': song.song_id})
            title = et.SubElement(song_info, 'title')
            title.text = song.title
            artist = et.SubElement(song_info, 'artist')
            artist.text = song.artist
            return et.tostring(song_info, encoding='unicode')
        else:
            raise ValueError(format)

Demo of its use

In [ ]:
song = Song('1', 'Water of Love', 'Dire Straits')

In [9]:
serializer = SongSerializer()

Create the JSON version of our object

In [10]:
serializer.serialize(song, 'JSON')

'{"id": "1", "title": "Water of Love", "artist": "Dire Straits"}'

Create the XML verrsion of our object

In [12]:
serializer.serialize(song, 'XML')

'<song id="1"><title>Water of Love</title><artist>Dire Straits</artist></song>'

This method of doing things is bad because:
    
- **SongSerializer** is doing more than 1 thing (that goes against the [single responsibility principle](https://en.wikipedia.org/wiki/Single_responsibility_principle))
- because of the **if/then/else**, it's hard to read and hard to maintain

Problems will show up if:
- a new format is introduced
- the Song object changes
- when the string representation ('JSON' vs 'JSON API') changes

**Ideal solution**

If any of those cases show up, we do not need to change the **serialize()** method.

# Looking for common Interface 

1. We need to find the common interface in our software. Here, the common goal is **to take a Song object and output a string**

2. We will provide separate implementations for each logical paths

3. Another **separate component** will decide of the concrete implementation to use



# Refactoring Code inoto the Desired Interface 

In [13]:
class SongSerializer:
    def serialize(self, song, format):
        if format == 'JSON':
            return self._serialize_to_json(song)
        elif format == 'XML':
            return self._serialize_to_xml(song)
        else:
            raise ValueError(format)
            
    def _serialize_to_json(self, song):
            song_info = {
                'id': song.song_id,
                'title': song.title,
                'artist': song.artist
            }
            return json.dumps(song_info)

    def _serialize_to_xml(self, song):
            song_info = et.Element('song', attrib={'id': song.song_id})
            title = et.SubElement(song_info, 'title')
            title.text = song.title
            artist = et.SubElement(song_info, 'artist')
            artist.text = song.artist
            return et.tostring(song_info, encoding='unicode')


In [15]:
song = Song('1', 'Water of Love', 'Dire Straits')
serializer = SongSerializer()
print("JSON: {}".format(serializer.serialize(song, 'JSON')))
print("XML: {}".format(serializer.serialize(song, 'XML')))

JSON: {"id": "1", "title": "Water of Love", "artist": "Dire Straits"}
XML: <song id="1"><title>Water of Love</title><artist>Dire Straits</artist></song>


This is better (and this is what I use to do to be honest) but this can still be improved by using the **Factory** method.

# Basic Implementation of Factory Method

In [18]:
class SongSerializer:
    
    def _get_serializer(self, format):
        """this method evaluates the value of the format and return the matching serialization function
        
        name: creator
        """
        if format == 'JSON':
            return self._serialize_to_json
        elif format == 'XML':
            return self._serialize_to_xml
        else:
            raise ValueError(format)
    
    def serialize(self, song, format):
        """name: client of the component pattern"""
        _serializer = self._get_serializer(format)
        return _serializer(song)
            
    def _serialize_to_json(self, song):
        """name: product component"""
            song_info = {
                'id': song.song_id,
                'title': song.title,
                'artist': song.artist
            }
            return json.dumps(song_info)

    def _serialize_to_xml(self, song):
        """name: product component"""
            song_info = et.Element('song', attrib={'id': song.song_id})
            title = et.SubElement(song_info, 'title')
            title.text = song.title
            artist = et.SubElement(song_info, 'artist')
            artist.text = song.artist
            return et.tostring(song_info, encoding='unicode')

In [19]:
song = Song('1', 'Water of Love', 'Dire Straits')
serializer = SongSerializer()
print("JSON: {}".format(serializer.serialize(song, 'JSON')))
print("XML: {}".format(serializer.serialize(song, 'XML')))

JSON: {"id": "1", "title": "Water of Love", "artist": "Dire Straits"}
XML: <song id="1"><title>Water of Love</title><artist>Dire Straits</artist></song>


More cleaning due to the fact that some of the class do not have to be part of the class

In [20]:
class SongSerializer:
    def serialize(self, song, format):
        serializer = get_serializer(format)
        return serializer(song)


def get_serializer(format):
    if format == 'JSON':
        return _serialize_to_json
    elif format == 'XML':
        return _serialize_to_xml
    else:
        raise ValueError(format)


def _serialize_to_json(song):
    payload = {
        'id': song.song_id,
        'title': song.title,
        'artist': song.artist
    }
    return json.dumps(payload)


def _serialize_to_xml(song):
    song_element = et.Element('song', attrib={'id': song.song_id})
    title = et.SubElement(song_element, 'title')
    title.text = song.title
    artist = et.SubElement(song_element, 'artist')
    artist.text = song.artist
    return et.tostring(song_element, encoding='unicode')

In [21]:
song = Song('1', 'Water of Love', 'Dire Straits')
serializer = SongSerializer()
print("JSON: {}".format(serializer.serialize(song, 'JSON')))
print("XML: {}".format(serializer.serialize(song, 'XML')))

JSON: {"id": "1", "title": "Water of Love", "artist": "Dire Straits"}
XML: <song id="1"><title>Water of Love</title><artist>Dire Straits</artist></song>
